In [ ]:
# !nvidia-smi

In [ ]:
from retina_function import *

In [ ]:
%matplotlib inline
import os
from glob import glob
from natsort import natsorted
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Input, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision, Accuracy
import numpy as np
import pandas as pd
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from tensorflow.keras.optimizers import Adam
from skimage import color
from skimage.filters import threshold_minimum, threshold_otsu
from livelossplot import PlotLossesKeras  

In [ ]:
config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config = config)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

### Creating mask from annotated images.
### creating folders to hold the mask, and associated images for training, validation and test dataset respectively.

##### 1 . Messidor data.

In [ ]:
folder_path = os.getcwd()
documents_direc = folder_path

# get the images from Messidor dataset
retina_images_2 = glob(os.path.join(documents_direc,"data","MESSIDOR","image*prime.tif"))
retina_masks_2= glob(os.path.join(documents_direc,"data","MESSIDOR","image*-6.tif"))

# get the images from BinRushed1 dataset
retina_images_1 = glob(os.path.join(documents_direc,"data","BinRushed","BinRushed1-Corrected","image*prime.jpg"))
retina_masks_1 = glob(os.path.join(documents_direc,"data","BinRushed","BinRushed1-Corrected","image*-6.jpg"))

# merge them together.
retina_images_2.extend(retina_images_1)
retina_masks_2.extend(retina_masks_1)



In [ ]:
# getting the path of the retina images and ground truth
retina_images = natsorted(retina_images_2)
retina_masks = natsorted(retina_masks_2)


print ("Total retina_images: {}, and total ground truth images: {}".
       format(len(retina_images),len(retina_masks)))

# split the retina_images into train and test images
train_ret, test_ret, train_mask, test_mask = train_test_split(
       retina_images, retina_masks, test_size=0.1, random_state=56)
# further split the train_retina images and train_mask into validation data.
# Thus we have both train, validation and test data, making our solution robust.

train_ret_, validation_ret, train_mask_, validation_mask = train_test_split(
       train_ret, train_mask, test_size=0.25, random_state=56)

train_mask_ = natsorted(train_mask_)
train_ret_ = natsorted(train_ret_)

validation_mask = natsorted(validation_mask)
validation_ret = natsorted(validation_ret)

test_mask = natsorted(test_mask)
test_ret = natsorted(test_ret)

# create a new folder to hold the train, test and validation retina_images.
retina_train_folder = os.path.join(documents_direc,"data","MESSIDOR","train/retina")
retina_valid_folder = os.path.join(documents_direc,"data","MESSIDOR","valid/retina")
retina_test_folder = os.path.join(documents_direc,"data","MESSIDOR","test/retina")

# create a new folder to hold the optic disc of the train, test and validation retina images
############################################################################################
mask_train_folder_disc = os.path.join(documents_direc,"data","MESSIDOR","mask_train/disc/retina")
mask_valid_folder_disc = os.path.join(documents_direc,"data","MESSIDOR","mask_valid/disc/retina")
mask_test_folder_disc = os.path.join(documents_direc,"data","MESSIDOR","mask_test/disc/retina")

# create a new folder to hold the optic cup of the train, test and validation retina images
############################################################################################
mask_train_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","mask_train/cup/retina")
mask_valid_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","mask_valid/cup/retina")
mask_test_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","mask_test/cup/retina")

# call the function to create the folder.
create_save_img(train_ret_, retina_train_folder) # train images and folder.
create_save_img(validation_ret, retina_valid_folder) # validation images and folder.
create_save_img(test_ret, retina_test_folder) # test images and folder.

# call the function to create mask for optic disc
###########################################################################
create_save_optic_disc(train_mask_, mask_train_folder_disc) # train mask and folder.
create_save_optic_disc(validation_mask, mask_valid_folder_disc) # validation mask and folder.
create_save_optic_disc(test_mask, mask_test_folder_disc) # test mask and folder.

# call the function to create mask for optic disc
###########################################################################
create_save_optic_cup(train_mask_, mask_train_folder_cup) # train mask and folder.
create_save_optic_cup(validation_mask, mask_valid_folder_cup) # validation mask and folder.
create_save_optic_cup(test_mask, mask_test_folder_cup) # test mask and folder.

In [ ]:
# print the total number of train, validation and test images along with their masks.
print ("Total retina train images: {}, and total ground truth images: {}".
       format(len(train_ret_),len(train_mask_)))
print ("Total retina validation images: {}, and total ground truth images: {}".
       format(len(validation_ret),len(validation_mask)))
print ("Total retina test images: {}, and total ground truth images: {}".
       format(len(test_ret),len(test_mask)))

train = natsorted(glob(os.path.join(retina_train_folder,"*.*")))
valid = natsorted(glob(os.path.join(retina_valid_folder,"*.*")))
test = natsorted(glob(os.path.join(retina_test_folder,"*.*")))

train_mask_cup = natsorted(glob(os.path.join(mask_train_folder_cup,"*.*")))
train_mask_disc = natsorted(glob(os.path.join(mask_train_folder_disc,"*.*")))

valid_mask_cup = natsorted(glob(os.path.join(mask_valid_folder_cup,"*.*")))
valid_mask_disc = natsorted(glob(os.path.join(mask_valid_folder_disc,"*.*")))

test_mask_cup = natsorted(glob(os.path.join(mask_test_folder_cup,"*.*")))
test_mask_disc = natsorted(glob(os.path.join(mask_test_folder_disc,"*.*")))

# processed;
roi_1, _ = get_roi(cv.imread(train_mask_[1]))
roi_1 = resize_rgb(roi_1)

roi_2, _ = get_roi(cv.imread(validation_mask[2]))
roi_2 = resize_rgb(roi_2)

roi_3, _ = get_roi(cv.imread(test_mask[1]))
roi_3 = resize_rgb(roi_3)


fig, ax = plt.subplots(nrows=6, ncols=3, figsize=(14,18))
show_image(ax[0,0],train_ret_[1],"Retina Image (train)")
show_image(ax[0,1],validation_ret[2],"Retina Image (validation)")
show_image(ax[0,2],test_ret[1],"Retina Image (test)")

show_image(ax[1,0],train_mask_[1],"Mask Image (train)")
show_image(ax[1,1],validation_mask[2],"Mask Image (validation)")
show_image(ax[1,2],test_mask[1],"Mask Image (test)")

show_image_roi(ax[2,0],roi_1,"Mask Image (train)")
show_image_roi(ax[2,1],roi_2,"Mask Image (validation)")
show_image_roi(ax[2,2],roi_3,"Mask Image (test)")

show_image(ax[3,0],train[4],"Reg. of Interest(train)")
show_image(ax[3,1],valid[4],"Reg. of Interest (validation)")
show_image(ax[3,2],test[2],"Reg. of Interest(test)")

show_image(ax[4,0],train_mask_disc[4],"Optic Disc(train)")
show_image(ax[4,1],valid_mask_disc[4],"Optic Disc (validation)")
show_image(ax[4,2],test_mask_disc[2],"Optic Disc (test)")

show_image(ax[5,0],train_mask_cup[4],"Optic Cup (train)")
show_image(ax[5,1],valid_mask_cup[4],"Optic Cup (validation)")
show_image(ax[5,2],test_mask_cup[2],"Optic Cup (test)")


plt.suptitle("Retinal Images, Corresponding Mask / Ground Truth Images and Region of Interest", fontsize="14")
plt.tight_layout()
plt.show()


orig, optic_disc, optic_cup = get_images(train, train_mask_disc, train_mask_cup, 
                                         retina_train_folder, mask_train_folder_disc, mask_train_folder_cup)

orig, optic_disc, optic_cup = get_images(valid, valid_mask_disc, valid_mask_cup,
                                       retina_valid_folder, mask_valid_folder_disc, mask_valid_folder_cup)

orig, optic_disc, optic_cup = get_images(test, test_mask_disc, test_mask_cup,
                                       retina_test_folder, mask_test_folder_disc, mask_test_folder_cup)

train = natsorted(glob(os.path.join(retina_train_folder,"*.*")))
valid = natsorted(glob(os.path.join(retina_valid_folder,"*.*")))
test = natsorted(glob(os.path.join(retina_test_folder,"*.*")))

train_mask_cup = natsorted(glob(os.path.join(mask_train_folder_cup,"*.*")))
train_mask_disc = natsorted(glob(os.path.join(mask_train_folder_disc,"*.*")))

valid_mask_cup = natsorted(glob(os.path.join(mask_valid_folder_cup,"*.*")))
valid_mask_disc = natsorted(glob(os.path.join(mask_valid_folder_disc,"*.*")))

test_mask_cup = natsorted(glob(os.path.join(mask_test_folder_cup,"*.*")))
test_mask_disc = natsorted(glob(os.path.join(mask_test_folder_disc,"*.*")))

# processed;
roi_1, _ = get_roi(cv.imread(train_mask_[1]))
roi_1 = resize_rgb(roi_1)

roi_2, _ = get_roi(cv.imread(validation_mask[2]))
roi_2 = resize_rgb(roi_2)

roi_3, _ = get_roi(cv.imread(test_mask[1]))
roi_3 = resize_rgb(roi_3)


fig, ax = plt.subplots(nrows=6, ncols=3, figsize=(14,18))
show_image(ax[0,0],train_ret_[1],"Retina Image (train)")
show_image(ax[0,1],validation_ret[2],"Retina Image (validation)")
show_image(ax[0,2],test_ret[1],"Retina Image (test)")

show_image(ax[1,0],train_mask_[1],"Mask Image (train)")
show_image(ax[1,1],validation_mask[2],"Mask Image (validation)")
show_image(ax[1,2],test_mask[1],"Mask Image (test)")

show_image_roi(ax[2,0],roi_1,"Mask Image (train)")
show_image_roi(ax[2,1],roi_2,"Mask Image (validation)")
show_image_roi(ax[2,2],roi_3,"Mask Image (test)")

show_image(ax[3,0],train[4],"Reg. of Interest(train)")
show_image(ax[3,1],valid[4],"Reg. of Interest (validation)")
show_image(ax[3,2],test[2],"Reg. of Interest(test)")

show_image(ax[4,0],train_mask_disc[4],"Optic Disc(train)")
show_image(ax[4,1],valid_mask_disc[4],"Optic Disc (validation)")
show_image(ax[4,2],test_mask_disc[2],"Optic Disc (test)")

show_image(ax[5,0],train_mask_cup[4],"Optic Cup (train)")
show_image(ax[5,1],valid_mask_cup[4],"Optic Cup (validation)")
show_image(ax[5,2],test_mask_cup[2],"Optic Cup (test)")


plt.suptitle("Retinal Images, Corresponding Mask / Ground Truth Images and Region of Interest", fontsize="14")
plt.tight_layout()
plt.show()

In [ ]:
# show the data plots;
"""
This function plots the class distribution of the dataset.
This is done for two reasons -> To check if the dataset is imbalance.
2 -> To check if we have enough dataset to train and also validate the model before deploying

:param axis represents the image to be plotted.
:param original represents the original images.
:param masked represents the masked images
:param title represents the title of the bar plot.
:param label_original represents the label for the original images
:param label_masks represents the label for the masked images
"""

def show_class_distribution(axis, original, masked_disc, masked_cup, title, label_original, label_masked_disc, label_masked_cup): 
    axis.bar("Original", original, color='r', label= label_original, width=0.7)
    axis.bar("Masked Disc", masked_disc, color='g', label= label_masked_disc, width=0.7)
    axis.bar("Masked Cup", masked_cup, color='y', label = label_masked_cup, width=0.7)
    axis.set_title(title, fontsize=14, fontweight='bold')
    axis.set_ylabel("length (arbs)", fontsize=13, fontweight='bold')
    axis.set_ylim((0,len(retina_images)+50))
    legend = axis.legend()
    

    ## checking after splitting.
fig, ax = plt.subplots(nrows=2,ncols=2, figsize = (16,12))
show_class_distribution(ax[0,0], len(train), len(train_mask_disc),len(train_mask_cup), 
                        "Class distribution for Train Data Set", "Original", "Masked Disc","Masked cup")
show_class_distribution(ax[0,1], len(valid), len(valid_mask_disc),len(valid_mask_cup), 
                        "Class distribution for Valid Data Set", "Original", "Masked Disc","Masked cup")
show_class_distribution(ax[1,0], len(test), len(test_mask_disc),len(test_mask_cup), 
                        "Class distribution for Test Data Set", "Original", "Masked Disc","Masked cup")
show_class_distribution(ax[1,1], len(retina_images), len(retina_masks),len(retina_masks), 
                        "Class distribution before Split", "Original", "Masked Disc","Masked cup")

plt.suptitle("Checking for balance between actual image and masked data", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# get the length of the datasets
print("length of training dataset is: {}, and training mask disc is {}, and training mask cup is {}".
      format(len(train), len(train_mask_disc), len(train_mask_cup)))
print("length of validation dataset is: {}, and validation mask disc is {}, and validation mask cup is {}".
      format(len(valid), len(valid_mask_disc), len(valid_mask_cup)))
print("length of test dataset is: {}, and test mask disc is {}, and test mask cup is {}".
      format(len(test), len(test_mask_disc), len(test_mask_cup)))

In [ ]:
# data preprocessing- normalizing the data for both train and validation set
# modified from: https://stackoverflow.com/questions/58050113/imagedatagenerator-for-semantic-segmentation

seed = 56 # to transform image and corresponding mask with same augmentation parameter.
IMG_SIZE = (400,400)
batch_size = 6
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    height_shift_range=0.2,
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    height_shift_range=0.2,
)


train_image_generator = train_datagen.flow_from_directory(
    os.path.join(documents_direc,"data","MESSIDOR","train"), 
    target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, subset="training", color_mode="rgb"
)

train_mask_generator_disc = train_datagen.flow_from_directory(
    os.path.join(documents_direc,"data","MESSIDOR","mask_train","disc")
    , target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, color_mode="grayscale"
)

train_mask_generator_cup = train_datagen.flow_from_directory(
    os.path.join(documents_direc,"data","MESSIDOR","mask_train","cup")
    , target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, color_mode="grayscale"
)

val_image_generator = train_datagen.flow_from_directory(
    os.path.join(documents_direc,"data","MESSIDOR","valid"), 
    target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, color_mode="rgb"
)

val_mask_generator_disc = train_datagen.flow_from_directory(
        os.path.join(documents_direc,"data","MESSIDOR","mask_valid","disc"), 
    target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, color_mode="grayscale"
)

val_mask_generator_cup = train_datagen.flow_from_directory(
        os.path.join(documents_direc,"data","MESSIDOR","mask_valid","cup"), 
    target_size=IMG_SIZE,batch_size= batch_size, class_mode=None,
    seed=seed, shuffle=False, color_mode="grayscale"
)

# zip both train and validation validators into a single file.
train_generator_disc = zip(train_image_generator, train_mask_generator_disc)
val_generator_disc = zip(val_image_generator, val_mask_generator_disc)
train_generator_cup = zip(train_image_generator, train_mask_generator_cup)
val_generator_cup = zip(val_image_generator, val_mask_generator_cup)

### Model Training

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# create directory to log the model and csv files to
log_file = os.path.join(documents_direc,"data","MESSIDOR","files")
create_directory(log_file)

# set the model hyperparamteres
BATCH = 6
learning_rate = 7e-5
num_epochs = 200
model_path_disc = log_file+"/retina_model_disc.h5"
csv_path_disc = log_file+"/data_disc.csv"

model_path_cup = log_file+"/retina_model_cup.h5"
csv_path_cup = log_file+"/data_cup.csv"



train_steps = (len(train_ret_)//BATCH)
validation_steps = (len(validation_ret)//BATCH)
input_shape = (400,400,3)


retina_model_disc = build_unet_model(input_shape)
retina_model_disc.summary()


In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# retinal_cup model
retina_model_cup = build_unet_model(input_shape)
retina_model_cup.summary()

metrics = [dice_coef, iou]

retina_model_disc.compile(
    loss=[dice_loss], optimizer=Adam(learning_rate), metrics=metrics
)

callbacks = [
    ModelCheckpoint(model_path_disc, verbose=1, save_best_only=True),
    CSVLogger(csv_path_disc),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
    PlotLossesKeras()
]

history = retina_model_disc.fit(
        train_generator_disc,
        epochs=num_epochs,
        validation_data=val_generator_disc,
        steps_per_epoch=train_steps,
        validation_steps=validation_steps,
        callbacks=callbacks
    )

In [ ]:
metrics = [dice_coef, iou]

retina_model_cup.compile(
    loss=[dice_loss], optimizer=Adam(learning_rate), metrics=metrics
)

callbacks = [
    ModelCheckpoint(model_path_cup, verbose=1, save_best_only=True),
    CSVLogger(csv_path_cup),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
    PlotLossesKeras()
]

history_cup = retina_model_cup.fit(
        train_generator_cup,
        epochs=num_epochs,
        validation_data=val_generator_cup,
        steps_per_epoch=train_steps,
        validation_steps=validation_steps,
        callbacks=callbacks
    )

# summarize history for accuracy
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice coeficient')
plt.ylabel('dice coeficient')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model Dice loss')
plt.ylabel('Dice loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')


In [ ]:
# summarize history for accuracy
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history_cup.history['dice_coef'])
plt.plot(history_cup.history['val_dice_coef'])
plt.title('model dice coeficient', fontsize=14, fontweight='bold')
plt.ylabel('dice coeficient', fontsize=14, fontweight='bold')
plt.xlabel('epoch', fontsize=14, fontweight='bold')
plt.legend(['train', 'validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history_cup.history['loss'])
plt.plot(history_cup.history['val_loss'])
plt.title('model Dice loss', fontsize=14, fontweight='bold')
plt.ylabel('Dice loss', fontsize=14, fontweight='bold')
plt.xlabel('epoch', fontsize=14, fontweight='bold')
plt.legend(['train', 'validation'], loc='upper left')


In [ ]:

from skimage.segmentation import clear_border, mark_boundaries
    
def show_image_pred(axis, image, title):
       axis.imshow(image, cmap='gray')
       axis.axis('off')
       axis.set_title(title)
        
        
def pred_and_save(image_, image_mask, save_directory):
    scores = []
    images = []
    dict_score = {}
    os.chdir(save_directory)
    for a in range(len(image_)):
        image = cv.imread(image_[a])
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        img = cv.resize(image, (400,400))
        img_yuv = cv.cvtColor(img, cv.COLOR_RGB2YUV)
        clahe = cv.createCLAHE(clipLimit=1, tileGridSize=(2,2))
        img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
        img = cv.cvtColor(img_yuv, cv.COLOR_YUV2RGB)
        img_1 = cv.bilateralFilter(img, 9, 75, 75)
        img = img_1.copy()

        
        x = img/255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
        
        y_pred = retina_model_cup.predict(x)[0]
        y_pred = np.where(y_pred > 0.5, 1.0, 0.0)

        y_pred_ = y_pred.copy()
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = cv.resize(y_pred, (400,400))
        kernel = np.ones((65,65), np.uint8)
        y_pred = cv.morphologyEx(y_pred, cv.MORPH_CLOSE, kernel)
        y_pred = y_pred.astype(np.float32)
        cv.imwrite("prediction-"+str(a)+".jpg", y_pred)
        y_true = cv.imread(image_mask[a])
        y_true = cv.cvtColor(y_true,cv.COLOR_BGR2GRAY)
        y_true = y_true.astype(np.float32)
        y_true = y_true/255.
        score = dice_coef(y_true, y_pred.astype(np.float32), smooth=1)
        scores.append(float(score))
        images.append(y_pred)
        dict_score[a] = float(score)
    return np.mean(scores) * 100, dict_score, images, scores


# saving the result to a folder
result_train_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","result/train/cup")
result_valid_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","result/valid/cup")
result_test_folder_cup = os.path.join(documents_direc,"data","MESSIDOR","result/test/cup")

create_directory(result_train_folder_cup)
create_directory(result_valid_folder_cup)
create_directory(result_test_folder_cup)

empty_directory(result_train_folder_cup)
empty_directory(result_test_folder_cup)
empty_directory(result_valid_folder_cup)


# creating directory for optic cup messidor prediction
train_score, dict_list_train, images_train, scores_train = pred_and_save(train, train_mask_cup, result_train_folder_cup)
valid_score, dict_list_valid, images_valid, scores_valid = pred_and_save(valid, valid_mask_cup, result_valid_folder_cup)
test_score, dict_list_test, images_test, scores_test = pred_and_save(test, test_mask_cup, result_test_folder_cup)

#
dict_list_train= sorted(dict_list_train, reverse=True)[:4]
scores_train = sorted(scores_train, reverse=True)[:4]

dict_list_valid= sorted(dict_list_valid, reverse=True)[:4]
scores_valid = sorted(scores_valid, reverse=True)[:4]

dict_list_test= sorted(dict_list_test, reverse=True)[:4]
scores_test = sorted(scores_test, reverse=True)[:4]

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(8,10))
r,c = 0,0
for b in range(len(dict_list_train)):
    if c >= 4:
        c = 0
    show_image_pred(ax[0,c], images_train[dict_list_train[b]],
                    "Dice Score: "+str(round(scores_train[b], 2)))
    show_image_pred(ax[1,c], images_valid[dict_list_valid[b]],
                    "Dice Score: "+str(round(scores_valid[b], 2)))
    show_image_pred(ax[2,c], images_test[dict_list_test[b]],
                    "Dice Score: "+str(round(scores_test[b], 2)))

    c += 1

plt.tight_layout()
plt.show()
        

In [ ]:
# print(len(scores_train))
print(dict_list_train)
print(len(train))
print(len(images))

In [ ]:
print(dict_)


In [ ]:
# print(len(retina_model_disc.layers))

In [ ]:
print("Train Dice coeficient Score is: {:.2f}".format(train_score))
print("Valid Dice coeficient Score is: {:.2f}".format(valid_score))
print("Train Dice coeficient Score is: {:.2f}".format(test_score))

###### 